In [1]:
from functools import partial

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

In [2]:
dist = tfd.Normal(0., 3.)

dist.cdf(1.)

<tf.Tensor: shape=(), dtype=float32, numpy=0.63055867>

In [3]:
@tf.function(experimental_compile=True)
def f(x):
  y = x * 2.0
  return tf.gradients(y, x)

In [4]:
f(tf.Variable(10, dtype=tf.float32))

[<tf.Tensor: shape=(), dtype=float32, numpy=2.0>]

In [5]:
with tf.GradientTape() as tape:
  x = tf.Variable(2, dtype=tf.float32)
  y = x * 3
  print(tape.gradient(y, x))

tf.Tensor(3.0, shape=(), dtype=float32)


In [6]:
with tf.GradientTape() as tape:
  x = tf.Variable(2.0, dtype=tf.float32)
  y = x ** 4
  print(tape.gradient(y, x))

tf.Tensor(32.0, shape=(), dtype=float32)


In [7]:
@tf.function(autograph=False, experimental_compile=True)
def func(params):
  mu, sigma = params[0], params[1]
  
  dist = tfd.Normal(mu, sigma)
  
  prob = dist.prob(1.0)
  
  phi = dist.cdf(1.0)
  
  return tf.gradients(phi, params)

In [8]:
inp = tf.constant([1.0, 3.0], dtype=tf.float32)

In [9]:
func(inp)

[<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.13298076, -0.        ], dtype=float32)>]

In [10]:
@tf.function(experimental_compile=True)
def outer(x, mean):
  
  y = (x[0] - mean) ** 2.0
  
  return y, tf.gradients(y, x)[0]

In [12]:
fn_obj = partial(outer, mean = tf.constant(3.2))

In [13]:
@tf.function(experimental_compile=True)
def uni_obj(x):
  
  y = (x[0] - 3.2) ** 2.0
  
  return y, tf.gradients(y, x)[0]

In [14]:
res = tfp.optimizer.lbfgs_minimize(
  fn_obj,
  initial_position=tf.constant([1.99]),
  tolerance=1e-7
)

In [15]:
res.position

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.2], dtype=float32)>